# RANKING V3

## Leer Datos

In [58]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Cargar datos
data = pd.read_csv("df_peleas.csv")  
df = pd.DataFrame(data)

In [80]:
df.columns

Index(['Unnamed: 0', 'index', 'Peleador_A', 'Peleador_B', 'DATE', 'CATEGORY',
       'WINNER', 'METHOD', 'TIME', 'ROUND', 'KD_A', 'KD_B', 'SIG_STR_A',
       'SIG_STR_B', 'TD_PORC_A', 'TD_PORC_B', 'SUB_ATT_A', 'SUB_ATT_B',
       'REV_A', 'REV_B', 'CTRL_A', 'CTRL_B', 'TOTAL_STR_A_x', 'TOTAL_STR_A_y',
       'TOTAL_STR_B_x', 'TOTAL_STR_B_y', 'TD_A_x', 'TD_A_y', 'TD_B_x',
       'TD_B_y', 'STR_HEAD_A_x', 'STR_HEAD_A_y', 'STR_HEAD_B_x',
       'STR_HEAD_B_y', 'STR_BODY_A_x', 'STR_BODY_A_y', 'STR_BODY_B_x',
       'STR_BODY_B_y', 'STR_LEG_A_x', 'STR_LEG_A_y', 'STR_LEG_B_x',
       'STR_LEG_B_y', 'STR_DISTANCE_A_x', 'STR_DISTANCE_A_y',
       'STR_DISTANCE_B_x', 'STR_DISTANCE_B_y', 'STR_CLINCH_A_x',
       'STR_CLINCH_A_y', 'STR_CLINCH_B_x', 'STR_CLINCH_B_y', 'STR_GROUND_A_x',
       'STR_GROUND_A_y', 'STR_GROUND_B_x', 'STR_GROUND_B_y', 'TITLE_FIGHT',
       'WOMEN'],
      dtype='object')

## Convertir Fechas y Ordenar

In [62]:
# Convertir la columna DATE a formato datetime y ordenar de más antigua a más reciente
df["DATE"] = pd.to_datetime(df["DATE"], errors='coerce')  
df = df.sort_values(by="DATE", ascending=True)

## Calcular penalizaciones por antigüedad de la pelea

In [64]:
# Función para calcular la importancia del tiempo con caída gradual
def calcular_factor_tiempo(fecha_pelea):
    LAMBDA = 0.2  # Penalización más leve para peleas antiguas
    años_transcurridos = (datetime.today() - fecha_pelea).days / 365
    return np.exp(-LAMBDA * (años_transcurridos ** 1.1))

## Calcular el factor tiempo de pelea

In [66]:
# Función para calcular el tiempo de pelea
def calcular_tiempo(pelea):
    tiempo = pelea["TIME"]
    round_p = (pelea["ROUND"] - 1) * 5 * 60
    return round_p + tiempo

## Calcular factor método

In [68]:
# Función para obtener factor de método
def obtener_factor_metodo(metodo):
    factores = {
        'KO/TKO': 2.5,
        'Submission': 2.4,
        'Decision - Unanimous': 1.5,
        'Decision - Majority': 1,
        'Decision - Split': 0.85,
        "TKO - Doctor's Stoppage": 1.0
    }
    return factores.get(metodo, 1)

## Calcular factor Oponente

In [70]:
# Función para calcular el factor de oponente en un rango de 1 a 5
def calcular_factor_oponente(puntos_ganador, puntos_perdedor, ganador=True):
    if puntos_ganador == 0:
        return 1  # Evita división por cero
    
    ratio = (puntos_perdedor + 100) / (puntos_ganador + 100)
    ratio = max(0.5, min(ratio, 2.0))

    min_ratio = 0.5
    max_ratio = 2.0

    factor_oponente = 1 + 9 * ((np.sqrt(ratio) - np.sqrt(min_ratio)) / (np.sqrt(max_ratio) - np.sqrt(min_ratio)))

    return max(1, min(5, factor_oponente))

## Calcular Ranking

In [72]:
# Diccionario para rastrear la información de cada peleador
peleadores_historial = {}

# DataFrame para almacenar la información histórica de cada pelea
df_historico = []

# Proceso de cálculo de puntos y almacenamiento de información histórica
for i, pelea in df.iterrows():
    ganador = pelea["WINNER"]
    peleador_a = pelea["Peleador_A"]
    peleador_b = pelea["Peleador_B"]
    fecha_pelea = pelea["DATE"]
    
    # Inicializar peleadores si no existen
    for peleador in [peleador_a, peleador_b]:
        if peleador not in peleadores_historial:
            peleadores_historial[peleador] = {"Puntos": 0, "Peleas": 0, "Racha": 0, "Ultima_Pelea": None}

    # Extraer valores actuales antes de la pelea
    puntos_a = peleadores_historial[peleador_a]["Puntos"]
    puntos_b = peleadores_historial[peleador_b]["Puntos"]
    racha_a = peleadores_historial[peleador_a]["Racha"]
    racha_b = peleadores_historial[peleador_b]["Racha"]

    # Guardar información previa en el histórico
    df_historico.append([
        fecha_pelea, peleador_a, puntos_a, racha_a, peleadores_historial[peleador_a]["Peleas"], pelea["CATEGORY"], peleador_b, "Ganó" if ganador == 0 else "Perdió"
    ])
    df_historico.append([
        fecha_pelea, peleador_b, puntos_b, racha_b, peleadores_historial[peleador_b]["Peleas"], pelea["CATEGORY"], peleador_a, "Ganó" if ganador == 1 else "Perdió"
    ])

    # Calcular modificadores
    factorTiempo = calcular_factor_tiempo(fecha_pelea)
    factorM = obtener_factor_metodo(pelea["METHOD"])
    factor_oAW = calcular_factor_oponente(puntos_a, puntos_b, ganador=True)
    factor_oAL = calcular_factor_oponente(puntos_a, puntos_b, ganador=False)
    factor_oBW = calcular_factor_oponente(puntos_b, puntos_a, ganador=True)
    factor_oBL = calcular_factor_oponente(puntos_b, puntos_a, ganador=False)

    factorTitleWin = 3 if pelea["TITLE_FIGHT"] else 1
    factorTitleLoss = 0.3 if pelea["TITLE_FIGHT"] else 1

    # Modificación por racha
    racha_A = 1 + 0.15 * peleadores_historial[peleador_a]["Racha"]
    racha_B = 1 + 0.15 * peleadores_historial[peleador_b]["Racha"]

    tiempo_total = calcular_tiempo(pelea)
    t_pelea = 1500 if pelea["TITLE_FIGHT"] else 900

    factorTime = max(1, min(2, 2 - 0.5 * (tiempo_total / t_pelea)))
    penalizacionA = (10 * factorTitleLoss * factorTime * factorM * factor_oAL * 3 / max(1, racha_B))
    penalizacionB = (10 * factorTitleLoss * factorTime * factorM * factor_oBL * 3 / max(1, racha_A))

    # Aplicar cambios en puntos
    if ganador == 0:  # Gana A
        peleadores_historial[peleador_a]["Racha"] += 1
        peleadores_historial[peleador_b]["Racha"] = 0
        peleadores_historial[peleador_a]["Puntos"] += (10 * factorTitleWin * factorTime * factorM * factor_oAW * factorTiempo * racha_A)
        peleadores_historial[peleador_b]["Puntos"] -= penalizacionB * factorTiempo
    elif ganador == 1:  # Gana B
        peleadores_historial[peleador_b]["Racha"] += 1
        peleadores_historial[peleador_a]["Racha"] = 0
        peleadores_historial[peleador_a]["Puntos"] -= penalizacionA * factorTiempo
        peleadores_historial[peleador_b]["Puntos"] += (10 * factorTitleWin * factorTime * factorM * factor_oBW  * factorTiempo * racha_B)

    peleadores_historial[peleador_a]["Puntos"] = max(0, peleadores_historial[peleador_a]["Puntos"])
    peleadores_historial[peleador_b]["Puntos"] = max(0, peleadores_historial[peleador_b]["Puntos"])

    # Actualizar número de peleas
    peleadores_historial[peleador_a]["Peleas"] += 1
    peleadores_historial[peleador_b]["Peleas"] += 1
    peleadores_historial[peleador_a]["Ultima_Pelea"] = fecha_pelea
    peleadores_historial[peleador_b]["Ultima_Pelea"] = fecha_pelea

# Convertir el historial a DataFrame
df_historico = pd.DataFrame(df_historico, columns=["Fecha", "Peleador", "Puntos", "Racha", "Peleas", "Categoria", "Oponente", "Resultado"])


## Eliminar peleadores que no han peleado en más de 3 años

In [ ]:
# Filtrar peleadores inactivos
#df_historico["Fecha"] = pd.to_datetime(df_historico["Fecha"])
#fecha_limite = datetime.today() - timedelta(days=3*365)
#df_historico = df_historico[df_historico["Fecha"] >= fecha_limite]
# Convertir el diccionario a un DataFrame

## Mostrar Top 15 LibraxLibra

In [74]:
df_peleadores = pd.DataFrame.from_dict(peleadores_historial, orient="index")

# Ordenar por puntos y mostrar los 15 mejores
df_top15 = df_peleadores.sort_values(by="Puntos", ascending=False).head(15)

print(df_top15)

                            Puntos  Peleas  Racha Ultima_Pelea
Islam Makhachev        3237.050763      17     15   2025-02-01
Jon Jones              2797.502461      24     20   2024-11-23
Dricus Du Plessis      2066.763808       9      9   2025-02-15
Ilia Topuria           1745.672928       8      8   2024-11-02
Alex Pereira           1615.092182      10      5   2024-10-12
Merab Dvalishvili      1613.713513      14     12   2025-02-01
Sean O'Malley          1304.104663      13      0   2024-09-28
Belal Muhammad         1295.996266      19     11   2024-08-03
Tom Aspinall           1256.570217       9      3   2024-08-03
Valentina Shevchenko   1248.311273      17      2   2024-09-28
Charles Oliveira       1167.724423      34      1   2024-11-23
Aljamain Sterling      1112.991924      21      0   2024-12-14
Alexandre Pantoja      1105.691571      16      7   2024-12-14
Zhang Weili            1097.892123      12      5   2025-02-15
Alexander Volkanovski  1081.994250      16      0   202

## Mostrar Historial Peleador

In [33]:
# Mostrar el resultado
print(df_historico[df_historico["Peleador"]=="Ilia Topuria"])
print(peleadores_historial["Ilia Topuria"])

           Fecha      Peleador       Puntos  Racha  Peleas      Categoria  \
11255 2020-10-10  Ilia Topuria     0.000000      0       0  FEATHERWEIGHT   
11418 2020-12-05  Ilia Topuria     8.183038      1       1  FEATHERWEIGHT   
12013 2021-07-10  Ilia Topuria   112.135368      2       2  FEATHERWEIGHT   
12689 2022-03-19  Ilia Topuria   217.301901      3       3    LIGHTWEIGHT   
13483 2022-12-10  Ilia Topuria   251.239738      4       4  FEATHERWEIGHT   
14017 2023-06-24  Ilia Topuria   359.842077      5       5  FEATHERWEIGHT   
14675 2024-02-17  Ilia Topuria   407.239455      6       6  FEATHERWEIGHT   
15422 2024-11-02  Ilia Topuria  1472.444461      7       7  FEATHERWEIGHT   

                    Oponente Resultado  
11255          Youssef Zalal      Ganó  
11418          Damon Jackson      Ganó  
12013              Ryan Hall      Ganó  
12689            Jai Herbert      Ganó  
13483         Bryce Mitchell      Ganó  
14017            Josh Emmett      Ganó  
14675  Alexander Vo